In [7]:
import requests
import re
import pandas as pd
import folium
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import json

In [8]:
#zipcodes in LA
f = open('inputZips.txt', 'r')
laZips = [z.replace('\n','') for z in f.readlines()]

In [9]:
def processResponse(r):
    #parse out each store's info
    stores = re.findall(r'"storeNumber":.*?"slug"', r)
    storeInfo = []
    for store in stores:
        #parse out info about each store
        storeInfo.append(list(re.findall(r'"storeNumber":"(.*?)".*?"name":"(.*?)".*?"latitude":(.*?),.*?"longitude":(.*?)}.*?"city":"(.*?)".*?"countrySubdivisionCode":"(.*?)".*?"postalCode":"(.*?)"', store)[0]))
    return storeInfo

In [10]:
allStores = []
for idx,z in enumerate(laZips):
    if idx%10 == 0:
        print(idx,'/',len(laZips))
    #search for 100 stores centered in given zip
    r = requests.get('https://www.starbucks.com/store-locator?map=34.216464,-118.656046,11z&place='+z)
    if r.status_code != 200:
        raise SystemExit
    storeInfoList = processResponse(r.text)
    for storeInfo in storeInfoList:
        storeInfo[6] = storeInfo[6][:5]
    allStores += storeInfoList

0 / 1


In [11]:
#construct non-duplicated list of stores
seenStoreIds = []
laStores = []
for store in allStores:
    if store[0] in seenStoreIds:
        continue
    else:
        laStores.append(store)
        seenStoreIds.append(store[0])

In [12]:
#define columns of csv
dfSbux = pd.DataFrame(columns=['id', 'strLocation', 'latitude', 'longitude', 'city', 'state', 'zip'])

#store data
for i,col in enumerate(dfSbux.columns):
    dfSbux[col] = [item[i] for item in laStores]

#cast the latitude and longitude as floats
dfSbux.latitude = dfSbux.latitude.apply(lambda x: float(x))
dfSbux.longitude = dfSbux.longitude.apply(lambda x: float(x))

#save to csv
dfSbux.to_csv('starbucksInUS.csv', index=False)

In [13]:
# dfSbux = pd.DataFrame(columns=['id', 'strLocation', 'latitude', 'longitude', 'city', 'state', 'zip'])
# for i,col in enumerate(dfSbux.columns):
#     dfSbux[col] = [item[i] for item in keepLAStores]

In [14]:
# dfSbux.latitude = dfSbux.latitude.apply(lambda x: float(x))
# dfSbux.longitude = dfSbux.longitude.apply(lambda x: float(x))

In [15]:
# laMap = folium.Map(location=[34.0522,-118.2437], tiles='Stamen Toner', zoom_start=10)
# for i,row in dfSbux.iterrows():
#     folium.CircleMarker((row.latitude,row.longitude), radius=3, weight=2, fill_color='red', fill_opacity=.9).add_to(laMap)

In [16]:
# folium.GeoJson(laArea).add_to(laMap)